In [0]:
!git clone https://github.com/ayan59dutta/election_parameters.git

fatal: destination path 'election_parameters' already exists and is not an empty directory.


In [0]:
import pandas as pd
import numpy as np

In [0]:
colmn_list = ['Candidate', 'Constituency', 'Criminal Case', 'Education', 'Total Assets', 'Liabilities']

def convert(x):
    return int(x.split()[1].replace(',', '').replace('~', ''))

df1_myneta_17 = pd.read_excel('election_parameters/data/MyNeta_UP_Assembly_ELections_2017.xlsx',
                          usecols=colmn_list,
                          converters={'Candidate': lambda x:x.upper(),
                                     'Total Assets': convert,
                                     'Liabilities': convert})

df1_myneta_17['Net Assets'] = df1_myneta_17['Total Assets'] - df1_myneta_17['Liabilities']
df1_myneta_17 = df1_myneta_17.drop(columns=['Total Assets', 'Liabilities'])
df1_myneta_17

,Candidate,Constituency,Criminal Case,Education,Net Assets
0,A HASIV,ARYA NAGAR,0,12th Pass,33578492
1,A WAHID,GAINSARI,0,10th Pass,75106
2,AAN SHIKHAR SHRIVASTAVA,GOSHAINGANJ,0,Graduate,41000
3,AAPTAB URF AFTAB,MUBARAKPUR,0,Illiterate,20000
4,AASHI GAUR,KHATAULI,0,Literate,3468543
5,AASHIF BEG,BARKHERA,0,Not Given,1330000
6,AATMA RAM,PURQAZI (SC),1,12th Pass,2326259
7,AATMARAM,TULSIPUR,0,Literate,782500
8,ABBAS AHMAD,MARIYAHU,1,Literate,1131432
9,ABBAS ALI ZAIDI URF RUSHDI MIYAN,RUDAULI,0,Graduate,20602366


In [0]:
colmn_list = ['ST_CODE', 'AC_NAME', 'AC_TYPE', 'CAND_NAME', 'CAND_SEX', 
              'CAND_CATEGORY', 'CAND_AGE', 'TOTVOTPOLL', 'POSITION']

df_ae12 = pd.read_excel('election_parameters/data/AE2012_8913.xls',
                        usecols=colmn_list,
                        header=1,
                        converters={'AC_NAME': lambda x: x.upper()})

df_ae12 = df_ae12[df_ae12.ST_CODE == 'S24'].drop('ST_CODE', axis=1).reset_index().drop('index', axis=1)

df_ae12

,AC_NAME,AC_TYPE,CAND_NAME,CAND_SEX,CAND_CATEGORY,CAND_AGE,TOTVOTPOLL,POSITION
0,BEHAT,GEN,MAHAVEER SINGH RANA,M,GEN,50,70274,1
1,BEHAT,GEN,NARESH,M,GEN,47,69760,2
2,BEHAT,GEN,UMAR ALI KHAN,M,GEN,34,47366,3
3,BEHAT,GEN,AJAY CHAUHAN,M,GEN,29,23623,4
4,BEHAT,GEN,DHARAM SINGH,M,SC,39,2251,5
5,BEHAT,GEN,DHANIRAM,M,GEN,39,1774,6
6,BEHAT,GEN,ARVIND,M,GEN,31,1399,7
7,BEHAT,GEN,KARI INAM,M,GEN,35,1308,8
8,BEHAT,GEN,YASHPAL,M,SC,28,781,9
9,BEHAT,GEN,ABDUL JABBAR,M,GEN,35,726,10


In [0]:
la_2017 = pd.read_excel('election_parameters/data/LA_2017.xls', usecols = [1, 6, 7, 8, 9,10, 11, 13, 14], converters={ 'AC_NAME': lambda x:x.upper()})
la_2017  = la_2017.drop(la_2017[((la_2017.ST_NAME != 'Uttar Pradesh'))].index)
la_2017 = la_2017.drop('ST_NAME', axis = 1)
la_2017.reindex()
la_2017